# HW1 - PM2.5 Prediction
- Preprocessing 
- Training 

In [ ]:
import os
import csv
import glob
import numpy as np
import pandas as pd

## Preprocessing

In [ ]:
def preprocess(df, feature):
    """Dataframe preprocess
    Preprocess the dataframe & extract specific value.

    Args:
        df: PM2.5 Dataframe
        feature: SO2, PM2.5, and so on

    Returns:
        processed dataframe

    """
    df = df[feature]
    df = df.str.extract('([0-9]+\.[0-9]+|[0-9]+)').astype('float32')
    df.fillna(value=0, inplace=True)
    df = np.array(df).flatten().reshape(1, -1)
    return df

def validation(inputData, outputLabel):
    """Data Cleansing
    
    Args:
        inputData: train_data
        outputLabel: train_label

    """
    if outputLabel < 0 or outputLabel > 65:
        return False
    for i in range(9):
        if inputData[i,10] < 0 or inputData[i,10] > 65:
            return False
    return True

In [ ]:
df = pd.read_csv('./datas/train_datas_1.csv')
features = df.columns.values.tolist()

isInit = False
for feature in features:
    series = preprocess(df, feature)
    if isInit is False:
        isInit = True
        train_dataset = series
    else:
        train_dataset = np.concatenate((train_dataset, series), axis=0)
        
train_dataset = train_dataset.transpose()

In [ ]:
nums = train_dataset.shape[0] - 9

isInit = False
for hr in range(nums):
    isValid = validation(train_dataset[hr:hr+9,:], train_dataset[hr+9, 10:11])
    if isValid == True:
        if isInit == False:
            train_datas = train_dataset[hr:hr+9,:].flatten()
            data_size = train_datas.shape[0]
            train_labels = train_dataset[hr+9, 10:11]
            isInit = True
        else:
            train_datas = np.concatenate((train_datas, train_dataset[hr:hr+9,:].flatten()))
            train_labels = np.concatenate((train_labels, train_dataset[hr+9, 10:11]))
            
train_datas = train_datas.reshape(-1, data_size)
train_labels = train_labels

# np.save('./hw1/train_datas.npy', train_datas)
# np.save('./hw1/train_labels.npy', train_labels)

## Model Training 

In [ ]:
X_train = train_datas
y_train = train_labels

# X_train = np.load('./hw1/train_datas.npy')
# y_train = np.load('./hw1/train_labels.npy')

epochs = 1e4
lr = 1e-3

w = np.zeros(X_train.shape[1])
s_grad = np.zeros(X_train.shape[1])

print ('[+] Model Training Start...')

for epoch in range(int(epochs)):
    y_pred = np.dot(X_train, w)
    loss = y_train - y_pred
    grad = np.dot(X_train.transpose(), loss) * (-2)
    s_grad += grad ** 2
    ada = np.sqrt(s_grad)
    w = w - lr * grad/ada
    if epoch % 500 == 0:
        mse =  (loss**2).mean(axis=None)
        print ('[{:04d}] - mse: {}\t'.format(epoch, mse))

np.save('./hw1/weights.npy', w)

print ('[+] Model Training Done...')